In [1]:
! pip install nsepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36084 sha256=c921483b071edd909700579a3ffc0d3e7638cd44b7a43c294a496875065cbba9
  Stored in directory: /root/.cache/pip/wheels/32/ab/d9/78ceea14cdf6de83376082b3cb0c2999fd77f823e35c47b9ec
Successfully built nsepy


In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 20, 20
import nsepy
df=pd.read_csv('/content/top100companies.csv')

In [17]:
 
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from matplotlib.backends.backend_pdf import PdfPages

In [4]:

from datetime import date, timedelta
data_s=nsepy.get_history('BSE',start=date.today()-timedelta(days=100),end=date.today()-timedelta(days=0))
data_s=data_s.drop(data_s.columns.difference(['VWAP']),axis=1)
data_s=data_s.rename(columns={'VWAP':'Sensex'})
df_s=data_s['Sensex']
sx=[]
for i in range(len(data_s.index)-1):
  price=(df_s[i+1]-df_s[i]) 
  if(price<0):
    sx.append(0)
  else:
    sx.append(1)
data_s=data_s.iloc[1:,:]
data_s['sx']=sx
data_s=data_s.drop('Sensex', axis=1)
data_s.head()



,sx
Date,
2022-03-30,1
2022-03-31,1
2022-04-01,0
2022-04-04,0
2022-04-05,0


In [5]:
df1=pd.DataFrame()
# nx=[]
for i in range(100):
  symbol=df['Symbol'][i]
  data_n=nsepy.get_history(symbol,start=date.today()-timedelta(days=100),end=date.today()-timedelta(days=0))
  data_n=data_n.drop(data_n.columns.difference(['VWAP']),axis=1)
  nx=[]
  for j in range(len(data_n.index)-1):
    price=( data_n['VWAP'][j+1]-data_n['VWAP'][j])*100/data_n['VWAP'][j+1]
    nx.append(price)
  data_n=data_n.iloc[1:,:]
  data_n[ symbol]=nx
  data_n=data_n.drop('VWAP', axis=1)
  df1=pd.concat([df1,data_n],axis=1)
df1.head()
   
 

,RELIANCE,TCS,HINDUNILVR,HDFCBANK,HDFC,INFY,KOTAKBANK,BHARTIARTL,ITC,ICICIBANK,...,WHIRLPOOL,HONAUT,TATAMOTORS,PNB,HINDALCO,GLAXO,3MINDIA,PEL,KANSAINER,ADANITRANS
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-30,1.441368,0.436705,1.234979,1.800452,2.430781,1.629018,1.247762,0.681917,-1.383258,1.995462,...,0.403570,2.595351,0.915634,-0.312856,-3.908779,1.175386,0.674925,1.092729,0.443479,-1.043182
2022-03-31,-0.732778,0.536674,1.430659,0.370426,1.053280,-0.132343,0.466579,-1.107696,-0.155261,0.669353,...,-0.502867,-0.076757,-0.845277,-0.113895,-5.038941,-0.028749,-0.084170,-1.125542,4.010046,-3.030981
2022-04-01,-0.019686,0.105140,1.222773,1.388211,0.735470,-0.503003,0.671489,1.137369,0.051727,0.060069,...,1.684449,0.084275,0.511883,2.579750,-1.414323,-0.435523,2.030132,1.084030,-1.219407,1.661621
2022-04-04,0.627876,0.525329,2.202176,9.474494,11.135032,-0.535945,1.706872,1.370630,1.148521,1.334860,...,1.760408,0.472022,1.944967,1.931447,1.908443,-1.638572,1.537034,1.652830,-0.285196,0.747724
2022-04-05,-0.806629,1.203239,1.535763,-1.944841,-3.162782,-0.708613,0.734942,-0.043014,1.799923,0.694231,...,2.426457,0.136675,2.514253,0.943142,1.034854,0.047667,3.848255,0.167795,1.281860,1.521806


In [6]:
co_data=pd.concat([df1,data_s],axis=1)
co_data.head()
 

,RELIANCE,TCS,HINDUNILVR,HDFCBANK,HDFC,INFY,KOTAKBANK,BHARTIARTL,ITC,ICICIBANK,...,HONAUT,TATAMOTORS,PNB,HINDALCO,GLAXO,3MINDIA,PEL,KANSAINER,ADANITRANS,sx
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-30,1.441368,0.436705,1.234979,1.800452,2.430781,1.629018,1.247762,0.681917,-1.383258,1.995462,...,2.595351,0.915634,-0.312856,-3.908779,1.175386,0.674925,1.092729,0.443479,-1.043182,1
2022-03-31,-0.732778,0.536674,1.430659,0.370426,1.053280,-0.132343,0.466579,-1.107696,-0.155261,0.669353,...,-0.076757,-0.845277,-0.113895,-5.038941,-0.028749,-0.084170,-1.125542,4.010046,-3.030981,1
2022-04-01,-0.019686,0.105140,1.222773,1.388211,0.735470,-0.503003,0.671489,1.137369,0.051727,0.060069,...,0.084275,0.511883,2.579750,-1.414323,-0.435523,2.030132,1.084030,-1.219407,1.661621,0
2022-04-04,0.627876,0.525329,2.202176,9.474494,11.135032,-0.535945,1.706872,1.370630,1.148521,1.334860,...,0.472022,1.944967,1.931447,1.908443,-1.638572,1.537034,1.652830,-0.285196,0.747724,0
2022-04-05,-0.806629,1.203239,1.535763,-1.944841,-3.162782,-0.708613,0.734942,-0.043014,1.799923,0.694231,...,0.136675,2.514253,0.943142,1.034854,0.047667,3.848255,0.167795,1.281860,1.521806,0


In [7]:
co_data.info()
co_data.isnull().values.any()
co_data=co_data.dropna(axis=1)
co_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 69 entries, 2022-03-30 to 2022-07-07
Columns: 101 entries, RELIANCE to sx
dtypes: float64(100), int64(1)
memory usage: 55.0+ KB


,RELIANCE,TCS,HINDUNILVR,HDFCBANK,HDFC,INFY,KOTAKBANK,BHARTIARTL,ITC,ICICIBANK,...,HONAUT,TATAMOTORS,PNB,HINDALCO,GLAXO,3MINDIA,PEL,KANSAINER,ADANITRANS,sx
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-30,1.441368,0.436705,1.234979,1.800452,2.430781,1.629018,1.247762,0.681917,-1.383258,1.995462,...,2.595351,0.915634,-0.312856,-3.908779,1.175386,0.674925,1.092729,0.443479,-1.043182,1
2022-03-31,-0.732778,0.536674,1.430659,0.370426,1.053280,-0.132343,0.466579,-1.107696,-0.155261,0.669353,...,-0.076757,-0.845277,-0.113895,-5.038941,-0.028749,-0.084170,-1.125542,4.010046,-3.030981,1
2022-04-01,-0.019686,0.105140,1.222773,1.388211,0.735470,-0.503003,0.671489,1.137369,0.051727,0.060069,...,0.084275,0.511883,2.579750,-1.414323,-0.435523,2.030132,1.084030,-1.219407,1.661621,0
2022-04-04,0.627876,0.525329,2.202176,9.474494,11.135032,-0.535945,1.706872,1.370630,1.148521,1.334860,...,0.472022,1.944967,1.931447,1.908443,-1.638572,1.537034,1.652830,-0.285196,0.747724,0
2022-04-05,-0.806629,1.203239,1.535763,-1.944841,-3.162782,-0.708613,0.734942,-0.043014,1.799923,0.694231,...,0.136675,2.514253,0.943142,1.034854,0.047667,3.848255,0.167795,1.281860,1.521806,0


In [ ]:
co_data.to_csv('company_price% data.csv')
from google.colab import files
files.download('company_price% data.csv')

In [ ]:
co_data.corr()

,RELIANCE,TCS,HINDUNILVR,HDFCBANK,HDFC,INFY,KOTAKBANK,BHARTIARTL,ITC,ICICIBANK,...,HONAUT,TATAMOTORS,PNB,HINDALCO,GLAXO,3MINDIA,PEL,KANSAINER,ADANITRANS,sx
RELIANCE,1.000000,0.341552,0.173508,0.230786,0.196924,0.355375,0.267827,0.351921,0.083188,0.373930,...,0.034103,0.559643,0.390719,0.440992,0.221986,0.373661,0.375074,0.282859,0.280026,0.344418
TCS,0.341552,1.000000,0.333086,0.506170,0.498426,0.767770,0.519096,0.458304,-0.002286,0.518450,...,0.190184,0.556627,0.358129,0.339847,0.180743,0.390520,0.275028,0.249194,0.240416,0.349362
HINDUNILVR,0.173508,0.333086,1.000000,0.390568,0.375034,0.364680,0.362327,0.305636,0.391060,0.417420,...,0.127759,0.487647,0.343623,0.222021,0.253830,0.249291,0.321662,0.308686,0.046644,0.393655
HDFCBANK,0.230786,0.506170,0.390568,1.000000,0.967749,0.508154,0.612620,0.497921,0.052719,0.498215,...,0.131597,0.395029,0.427932,0.285153,0.070087,0.160507,0.247947,0.104485,0.132173,0.279108
HDFC,0.196924,0.498426,0.375034,0.967749,1.000000,0.517592,0.618312,0.433500,0.068927,0.446009,...,0.080954,0.356452,0.350725,0.235546,0.070346,0.141051,0.193958,0.065917,0.093012,0.259614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3MINDIA,0.373661,0.390520,0.249291,0.160507,0.141051,0.346102,0.051173,0.303791,0.244446,0.308226,...,0.170398,0.450206,0.390446,0.347503,0.077391,1.000000,0.427861,0.235892,0.100099,0.204093
PEL,0.375074,0.275028,0.321662,0.247947,0.193958,0.334415,0.190963,0.441088,0.196777,0.483729,...,0.099205,0.504846,0.410694,0.442815,0.155799,0.427861,1.000000,0.151795,0.170090,0.261236
KANSAINER,0.282859,0.249194,0.308686,0.104485,0.065917,0.179536,0.288438,0.201711,0.149392,0.307165,...,0.193706,0.322269,0.323547,0.139596,0.234502,0.235892,0.151795,1.000000,0.314765,0.285174
ADANITRANS,0.280026,0.240416,0.046644,0.132173,0.093012,0.228977,0.226268,0.214340,0.255899,0.202369,...,0.132804,0.359206,0.390581,0.395082,0.290903,0.100099,0.170090,0.314765,1.000000,0.244843


In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = 20, 20
sns.heatmap(co_data.corr())

In [ ]:
bp = PdfPages('BoxPlots with sx Split.pdf')

for num_variable in co_data.columns:
    fig,axes = plt.subplots(figsize=(10,4))
    sns.boxplot(x='sx', y=num_variable, data = co_data)
    plt.title(str('Box Plot of ') + str(num_variable))
    bp.savefig(fig)
bp.close()

bp = PdfPages('BoxPlots with whole view.pdf')
for num_variable in co_data.columns:
  fig,axes = plt.subplots(figsize=(10,4))
  sns.boxplot(y=num_variable,data=co_data)
  plt.title(str('Box Plot of ') + str(num_variable))
  bp.savefig(fig)
bp.close()

In [8]:
train_features = co_data.columns.difference(['sx'])
train_X,test_X,train_y,test_y= train_test_split(co_data[train_features],co_data['sx'],test_size=0.3, random_state=42)
train_X.columns

Index(['3MINDIA', 'ADANIGREEN', 'ADANIPORTS', 'ADANITRANS', 'ALKEM',
       'AMBUJACEM', 'ASIANPAINT', 'AUROPHARMA', 'AXISBANK', 'BAJAJ-AUTO',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BANKBARODA', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CIPLA',
       'COALINDIA', 'COLPAL', 'DABUR', 'DIVISLAB', 'DLF', 'DMART', 'DRREDDY',
       'EICHERMOT', 'GAIL', 'GLAXO', 'GODREJCP', 'GRASIM', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDPETRO', 'HINDUNILVR', 'HINDZINC', 'HONAUT',
       'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL', 'INDIGO', 'INDUSINDBK',
       'INFY', 'IOC', 'ITC', 'JSWSTEEL', 'KANSAINER', 'KOTAKBANK', 'LT', 'LTI',
       'LUPIN', 'M&M', 'MARICO', 'MARUTI', 'MCDOWELL-N', 'MRF', 'MUTHOOTFIN',
       'NAUKRI', 'NMDC', 'NTPC', 'ONGC', 'PEL', 'PETRONET', 'PFC', 'PGHH',
       'PIDILITIND', 'PNB', 'POWERGRID', 'RELIANCE', 'SBICARD', 'SBILIFE',
       'SBIN', 'SHREECEM', 'SIEM

In [15]:
# logreg=sm.logit(formula=str('sx ~')+str(train_features),data=train_X)
# result=logreg.fit()
# summ=result.summary() 
logreg=LogisticRegression()
model=logreg.fit(train_X,train_y)
model.score(train_X,train_y)
model.coef_

array([[-0.00515482, -0.03520823,  0.13205871, -0.05501111,  0.04379255,
        -0.31766696, -0.04120898,  0.23758976, -0.19938122, -0.07534971,
         0.08005554,  0.22712275,  0.17890152, -0.01599024, -0.11903574,
        -0.08726574, -0.00345895,  0.19418572, -0.07616583,  0.11479797,
        -0.10414418, -0.01382218,  0.19530186,  0.17412431, -0.04107413,
        -0.1624558 ,  0.34325218,  0.22901749, -0.25543278, -0.00354631,
         0.08837477,  0.12039329,  0.06743659, -0.08534558,  0.07523924,
        -0.05045167, -0.28546023, -0.12637406, -0.00523287,  0.07733278,
        -0.16127077,  0.10325044,  0.30031678, -0.0857625 ,  0.3666952 ,
         0.01477919, -0.26726248,  0.26922279, -0.074851  ,  0.12119961,
         0.1999346 ,  0.2018961 , -0.22428602,  0.14123556,  0.33422527,
         0.1837049 , -0.01773023, -0.05785429,  0.28311757,  0.02002996,
        -0.02300683, -0.07140896,  0.00470944, -0.02011224, -0.15281317,
         0.03522641,  0.39863322,  0.04026578, -0.1

In [ ]:
train_predicted_prob = pd.DataFrame(model.predict(train_X))
train_predicted_prob.columns = ['prob']
train_actual = train_y
train_predict = pd.concat([train_actual, train_predicted_prob], axis=1)
train_predict.columns = ['actual','prob']
train_predict.head()

In [ ]:
test_predicted_prob = pd.DataFrame(result.predict(test_X))
test_predicted_prob.columns = ['prob']
test_actual = test_X['sx']
# making a DataFrame with actual and prob columns
test_predict = pd.concat([test_actual, test_predicted_prob], axis=1)
test_predict.columns = ['actual','prob']
test_predict.head()

In [24]:
y_pred = model.predict(test_X)
y_pred
# # model.score(train_X,train_y)
# model.score(test_X,test_y)
# # confusion_matrix(test_y, y_pred)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0])

In [19]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90        15
           1       0.80      0.67      0.73         6

    accuracy                           0.86        21
   macro avg       0.84      0.80      0.82        21
weighted avg       0.85      0.86      0.85        21

